# Basic Introduction to Functions and States

Using the functions and objects in `autora.state`, we can build flexible pipelines and cycles which operate on state
objects.

## Theoretical Overview

The fundamental idea is this:
- We define a "state" object $S$ which can be modified with a "delta" (a new result) $\Delta S$.
- A new state at some point $i+1$ is $$S_{i+1} = S_i + \Delta S_{i+1}$$
- The cycle state after $n$ steps is thus $$S_n = S_{0} +  \sum^{n}_{i=1} \Delta S_{i}$$

To represent $S$ and $\Delta S$ in code, you can use `autora.state.delta.State` and `autora.state.delta.Delta`
respectively. To operate on these, we define functions.

- Each operation in an AER cycle (theorist, experimentalist, experiment_runner, etc.) is implemented as a
function with $n$ arguments $s_j$ which are members of $S$ and $m$ others $a_k$ which are not.
  $$ f(s_0, ..., s_n, a_0, ..., a_m) \rightarrow \Delta S_{i+1}$$
- There is a wrapper function $w$ (`autora.state.delta.wrap_to_use_state`) which changes the signature of $f$ to
require $S$ and aggregates the resulting $\Delta S_{i+1}$
  $$w\left[f(s_0, ..., s_n, a_0, ..., a_m) \rightarrow \Delta
S_{i+1}\right] \rightarrow \left[ f^\prime(S_i, a_0, ..., a_m) \rightarrow S_{i} + \Delta
S_{i+1} = S_{i+1}\right]$$

- Assuming that the other arguments $a_k$ are provided by partial evaluation of the $f^\prime$, the full AER cycle can
then be represented as:
  $$S_n = f_n^\prime(...f_2^\prime(f_1^\prime(S_0)))$$

There are additional helper functions to wrap common experimentalists, experiment runners and theorists so that we
can define a full AER cycle using python notation as shown in the following example.

## Example

First initialize the State. In this case, we use the pre-defined `StandardState` which implements the standard AER
naming convention.
There are two variables `x` with a range [-10, 10] and `y` with an unspecified range.

In [ ]:
from autora.state.bundled import StandardState
from autora.variable import VariableCollection, Variable

s_0 = StandardState(
    variables=VariableCollection(
        independent_variables=[Variable("x", value_range=(-10, 10))],
        dependent_variables=[Variable("y")]
    )
)

Specify the experimentalist. Use a standard function `random_pool`.
This gets 5 independent random samples (by default, configurable using an argument)
from the value_range of the independent variables, and returns them in a DataFrame.
To make this work as a function on the State objects, we wrap it in the `on_state` function.

In [ ]:
from autora.experimentalist.random_ import random_pool
from autora.state.delta import on_state

experimentalist = on_state(function=random_pool, output=["conditions"])
s_1 = experimentalist(s_0, random_state=42)
s_1

StandardState(variables=VariableCollection(independent_variables=[Variable(name='x', value_range=(-10, 10), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=          x
0  5.479121
1 -1.222431
2  7.171958
3  3.947361
4 -8.116453, experiment_data=None, models=[])

Specify the experiment runner. This calculates a linear function, adds noise, assigns the value to the `y` column
 in a new DataFrame.

In [ ]:
from autora.state.delta import on_state
import numpy as np
import pandas as pd


@on_state(output=["experiment_data"])
def experiment_runner(conditions: pd.DataFrame, c=[2, 4], random_state = None):
    rng = np.random.default_rng(random_state)
    x = conditions["x"]
    noise = rng.normal(0, 1, len(x))
    y = c[0] + (c[1] * x) + noise
    observations = conditions.assign(y = y)
    return observations

# Which does the following:
experiment_runner(s_1, random_state=43)

StandardState(variables=VariableCollection(independent_variables=[Variable(name='x', value_range=(-10, 10), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=          x
0  5.479121
1 -1.222431
2  7.171958
3  3.947361
4 -8.116453, experiment_data=          x          y
0  5.479121  24.160713
1 -1.222431  -2.211546
2  7.171958  30.102304
3  3.947361  16.880769
4 -8.116453 -32.457650, models=[])

A completely analogous definition, using the separate `@inputs_from_state` and `@outputs_to_delta(...)` decorators
rather than the combined `@on_state(...)` decorator would be:

In [ ]:
from autora.state.delta import inputs_from_state, outputs_to_delta


@inputs_from_state
@outputs_to_delta("experiment_data")
def experiment_runner_alt_1(conditions: pd.DataFrame, c=[2, 4], random_state=None):
    x = conditions["x"]
    rng = np.random.default_rng(random_state)
    noise = rng.normal(0, 1, len(x))
    y = c[0] + (c[1] * x) + noise
    xy = conditions.assign(y = y)
    return xy

# Which does the following:
experiment_runner_alt_1(s_1, random_state=42)

StandardState(variables=VariableCollection(independent_variables=[Variable(name='x', value_range=(-10, 10), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=          x
0  5.479121
1 -1.222431
2  7.171958
3  3.947361
4 -8.116453, experiment_data=          x          y
0  5.479121  24.221201
1 -1.222431  -3.929709
2  7.171958  31.438285
3  3.947361  18.730007
4 -8.116453 -32.416847, models=[])

Or alternatively:

In [ ]:
def experiment_runner_alt_2_core(conditions: pd.DataFrame, c=[2, 4], random_state=None):
    x = conditions["x"]
    rng = np.random.default_rng(random_state)
    noise = rng.normal(0, 1, len(x))
    y = c[0] + (c[1] * x) + noise
    xy = conditions.assign(y = y)
    return xy

experiment_runner_alt_2 = on_state(experiment_runner_alt_2_core, output=["experiment_data"])
experiment_runner_alt_2(s_1)

StandardState(variables=VariableCollection(independent_variables=[Variable(name='x', value_range=(-10, 10), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=          x
0  5.479121
1 -1.222431
2  7.171958
3  3.947361
4 -8.116453, experiment_data=          x          y
0  5.479121  24.086818
1 -1.222431  -2.709502
2  7.171958  29.911578
3  3.947361  18.928439
4 -8.116453 -27.768580, models=[])

Specify a theorist, using a standard LinearRegression from scikit-learn.

In [ ]:
from sklearn.linear_model import LinearRegression
from autora.state.wrapper import state_fn_from_estimator

theorist = state_fn_from_estimator(LinearRegression(fit_intercept=True))

Now we can run the theorist on the output from the experiment_runner,
which itself uses the output from the experimentalist.

In [ ]:
theorist(experiment_runner(experimentalist(s_0)))

StandardState(variables=VariableCollection(independent_variables=[Variable(name='x', value_range=(-10, 10), allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], dependent_variables=[Variable(name='y', value_range=None, allowed_values=None, units='', type=<ValueType.REAL: 'real'>, variable_label='', rescale=1, is_covariate=False)], covariates=[]), conditions=          x
0 -5.213077
1  4.831915
2 -2.014685
3  4.923726
4 -4.931893, experiment_data=          x          y
0 -5.213077 -18.202418
1  4.831915  21.526622
2 -2.014685  -5.383766
3  4.923726  21.485098
4 -4.931893 -18.631364, models=[LinearRegression()])

If we like, we can run the experimentalist, experiment_runner and theorist ten times.

In [ ]:
s_ = s_0
for i in range(10):
    s_ = experimentalist(s_)
    s_ = experiment_runner(s_)
    s_ = theorist(s_)

The experiment_data has 50 entries (10 cycles and 5 samples per cycle):

In [ ]:
s_.experiment_data

,x,y
0,-9.169342,-34.642850
1,-3.688797,-11.369347
2,0.032322,1.283628
3,7.655542,32.158661
4,-2.020976,-6.004714
5,-9.856663,-37.083597
6,1.049356,5.327947
7,-5.753153,-20.917821
8,0.588991,5.126615
9,3.813722,18.092712


The fitted coefficients are close to the original intercept = 2, gradient = 4

In [ ]:
print(s_.model.intercept_, s_.model.coef_)


[2.04522595] [[4.03328388]]
